# Diagnostics of mock photometry

_Alex Malz (GCCL@RUB)_

In [ ]:
import corner
from itertools import combinations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import manifold

In [ ]:
toppath = '/media/CRP6/Cosmology/recidivator/'

In [ ]:
all_cols = ['lsstr-lssti', 'lssti-lsstz', 'lsstz-lssty']
filt = 'lsst'
bands = 'rizy'

In [ ]:
def prep_for_corner(dataset, cols):
    return np.array([dataset[col] for col in cols]).T

## start with examining GAMA data

In [ ]:
# gamapath = toppath+'GAMA/SpecObjPhot.csv'

In [ ]:
# data = pd.read_csv(gamapath)

In [ ]:
# data.columns

### divide by field

In [ ]:
# xlim = {}
# xlim['G02'] = (30.2, 38.8)
# xlim['G09'] = (129.0, 141.0)
# xlim['G12'] = (174.0, 186.0)
# xlim['G15'] = (211.5, 223.5)

# ylim = {}
# ylim['G02'] = (-10.5, -3.72)
# ylim['G09'] = (-2., 3.)
# ylim['G12'] = (-3., 2.)
# ylim['G15'] = (-2., 3.)

### isolate photometry and redshift

In [ ]:
# gama_fields ={}
# for key in xlim.keys():
#     gama_fields[key] = data.http://ls.st/cet-zoomloc[(data['RA'] > xlim[key][0]) & (data['RA'] < xlim[key][1]) & (data['DEC'] > ylim[key][0]) & (data['DEC'] < ylim[key][1])][['Z']+['lsst'+band for band in 'grizy']]
# #     gama_fields[key].to_csv(toppath+'GAMA/'+key+'SpecObjPhot.csv')

In [ ]:
# corner.corner(gama_fields['G15'], xlim=(8, 24), ylim=(8, 24))

In [ ]:
# for i in gama_fields['G15'].columns:
#     plt.hist(gama_fields['G15'][i], bins=100)
#     plt.xlabel(i)
#     plt.show()

this is where I should filter them by which of 'grizy' they have (not counting 0s and NaNs) and color by those later

In [ ]:
# for i in range(len('grizy')):
#     print(list(combinations('grizy', i+1)))

want a better visualization for high-D data

In [ ]:
# X = gama_fields['G15'].values

In [ ]:
# tsne = manifold.TSNE()
# res = tsne.fit_transform(X)

In [ ]:
# plt.scatter(res[:, 0], res[:, 1], s=1, alpha=0.5)
# plt.show()

### after environment classification

let's see if these problems are solved by looking at per-environment-class data, since that's what we'll be comparing regardless.

In [ ]:
gamapath = '/media/CRP6/Cosmology/workdir/emille/results_norm/15PCs/G09/'

In [ ]:
gamadata = pd.read_csv(gamapath+'classes_10_groups.csv')

In [ ]:
gamadata.columns

In [ ]:
# help(gamadata.notnull())

In [ ]:
cols = ['lsstr', 'lssti', 'lsstz', 'lssty']

fig, axs = plt.subplots(1, len(cols))
for i, col in enumerate(cols):
#     print((len(gamadata[col]), len(gamadata[col].dropna())))
    axs[i].hist(gamadata[col].dropna()[gamadata[col] != 0])
#     axs[i].hist(mockdata[col])
    axs[i].set_xlabel(col)

In [ ]:
# gama_for_corner = []
# for i in range(10):
#     gama_for_corner.append(prep_for_corner(gamadata[gamadata['label'] == i], all_cols))

In [ ]:
# #['lsstg-lsstr', 'lsstr-lssti', 'lssti-lsstz']
# for filt in ['lsst']:
#     bands = 'rizy'
#     for i, b in enumerate(bands[:-1]):
#         nonans = pd.notna(gamadata[filt+b] and gamadata[filt+bands[i+1]])
#         gamadata[filt+b+'-'+filt+bands[i+1]][nonans] = gamadata[filt+b][nonans] - gamadata[filt+bands[i+1]][nonans]
        
for i, b in enumerate(bands[:-1]):
    wherenotnaleft = ~(gamadata[filt+b] == -0)
    wherenotnaright = ~(gamadata[filt+bands[i+1]] == -0)
    whereboth = gamadata[[filt+b, filt+bands[i+1]]][(wherenotnaleft & wherenotnaright)]
    gamadata[filt+b+'-'+filt+bands[i+1]] = whereboth[filt+b] - whereboth[filt+bands[i+1]]

In [ ]:
# for i in range(10):
#     if i == 0:
#         fig = corner.corner(gama_for_corner[i], labels=all_cols)
#     else:
#         corner.corner(gama_for_corner[i], labels=all_cols)

## the mock photometry

In [ ]:
mockpath = '/media/CRP6/Cosmology/workdir/kara/results_LOS42_G09_10groups_15PCs.csv'

In [ ]:
mockdata = pd.read_csv(mockpath)

In [ ]:
mockdata.columns

In [ ]:
cols = ['mag_r_lsst', 'mag_i_lsst', 'mag_z_lsst', 'mag_y_lsst']

fig, axs = plt.subplots(1, len(cols))
for i, col in enumerate(cols):
#     print((len(mockdata[col]), len(mockdata[col].dropna())))
    axs[i].hist(mockdata[col].dropna()[mockdata[col] != -999])
#     axs[i].hist(mockdata[col])
    axs[i].set_xlabel(col)

In [ ]:
for i, b in enumerate(bands[:-1]):
    wherenotnaleft = ~(mockdata['mag_'+b+'_'+filt] == -999)
    wherenotnaright = ~(mockdata['mag_'+bands[i+1]+'_'+filt] == -999)
    whereboth = mockdata[['mag_'+b+'_'+filt, 'mag_'+bands[i+1]+'_'+filt]][(wherenotnaleft & wherenotnaright)]
    mockdata[filt+b+'-'+filt+bands[i+1]] = whereboth['mag_'+b+'_'+filt] - whereboth['mag_'+bands[i+1]+'_'+filt]

## next steps

* mag vs. redshift 
* color vs. redshift
* color vs. color

propensity scores (TODO: hunt down old code from stats class 2018)

In [ ]:
# fig, axs = plt.subplots(len(all_cols), 1)
# for i, col in enumerate(all_cols):
#     axs[i].hist(gamadata[col])
#     axs[i].hist(mockdata[col])
#     axs[i].set_xlabel(col)

In [ ]:
# def prep_for_corner(dataset, cols):
#     return np.array([dataset[col] for col in cols]).T

# gama_for_corner = prep_for_corner(gamadata, all_cols)
# mock_for_corner = prep_for_corner(mockdata, all_cols)
# fig = corner.corner(mock_for_corner, labels=all_cols)
# corner.corner(gama_for_corner, labels=all_cols)
# # for i, which_os in enumerate(available_os):
# #     if i == 0:
# #         fig = corner.corner(prep_for_corner(available_os[i], labels), labels=labels, alpha=0.25)
# #     else:
# #         corner.corner(prep_for_corner(which_os, labels), fig=fig, color=os_colors[which_os], alpha=0.25)
# #   # corner.overplot_points(fig, [float(metadata[which_os][i+2]) for i in range(6)], color=os_colors[which_os], alpha=0.5)
# #   # not sure why the overplotting of limits (as lines or points) fails given corner's documentation. . .